In [95]:
rootf = FOLDER_NAME ## Put the folder path where folder RBM-MHC is saved ##

## imports ##

import sys, os, pickle
sys.path.append(rootf + '/rbm/PGM3/source/')
sys.path.append(rootf + '/rbm/PGM3/utilities/')
from common_imports import *
import dataset_utils, plots_utils, RBM_utils, evaluate_learning_utils, Proteins_utils, sequence_logo
import utilities as utilities
import rbm as rbm
import importlib
import butils

We focus on sample 12T from Kalaora et al. 2016, Kalaora et al. 2018
We first visualize motifs given by peptides of the same HLA binding specificity as 
predicted by RBM-MHC, as in Fig. 2a and Supplementary Fig. 8.

In [96]:
clas = '12T'
out_fold = rootf + '/rbm-mhc/data/Melanoma_samples/' + clas
HLA_filename = out_fold + '/HLA-typing.txt'

hlas = []
with open(HLA_filename) as f:
    for line in f:
        linesplit = line.strip().split('\n')
        hlas.append(linesplit[0])
nc = len(hlas)

filename_motif = out_fold + '/full_sample_original_classification_YesReal_1_hu_10_l120.001_AL9.5_SA9_RWNoAlpha_TR1.txt'
table1 = pd.read_csv(filename_motif , sep='\t') ## extract data from original, patient-derived, dataset ##
table2 = table1[table1['TR']==0]

CC = 21 # number of symbols: amino acids + gap

SA = 9 # length of the alignment ('sequence core')
SAmin = 8 # minimal sequence length in the sample
SAmax = 11 # maximal sequence length in the sample
pseudocount = 0.000001

import sequence_logo

fig, axess = plt.subplots(6,1)
fig.set_figheight(7)
fig.set_figwidth(4)
fig.subplots_adjust(wspace=0.5, hspace = 1.1, bottom = 0.2)
st=12
sl=st

seqs_all=[]
seqs_orig=[]

for h in range(nc):
    seqs_origh = list(table2[table2['RBM'] == h]['Data Core'].values)
    seqs_allh = list(table1[table1['RBM'] == h]['Data Core'].values)
    Nrbm=len(seqs_allh)
    seqs_all.append(seqs_allh)
    seqs_orig.append(seqs_origh)
    sel_n = butils.convert_number(seqs_origh)
    pwm_rbm = utilities.average(sel_n, c = CC);
        
    for i in range(SA):
        for a in range(CC):
            if pwm_rbm[i,a] < pseudocount:
                pwm_rbm[i,a] = pseudocount  
            
    ax1= axess[h]
    titlep =  hlas[h] + ' motif (# seqs. = ' + str(Nrbm) + ')'
    sequence_logo.Sequence_logo(pwm_rbm, ax = ax1, figsize=(12,3.5), ylabel= 'bits', title = titlep, show=True, ticks_every=1, ticks_labels_size=sl, title_size=st)

plt.tight_layout()
namefig = out_fold + '/motifs_reconstructed.png'
fig.savefig(namefig,dpi=300)
plt.close()

Here, we upload the trained RBM model, we visualize its weights and examples of inputs to hidden units, similarly to Fig. 1b and Supplementary Figs. 3-4.

In [97]:
name_r = out_fold + '/model_YesReal_1_hu_10_l120.001_AL9.5_SA9_RWNoAlpha_TR1.data'
RBM = RBM_utils.loadRBM(name_r)
hu = RBM.n_h ## hidden units of the model ##

## figure with weights ## 
fig, axess = plt.subplots(10,1)
fig.set_figheight(11)
fig.set_figwidth(4)
fig.subplots_adjust(wspace=0.5, hspace = 1., bottom = 0.2)
st=12
sl=st

for h in range(hu):           
    ax1= axess[h]
    titlep = ''
    sequence_logo.Sequence_logo(RBM.weights[h], ax = ax1, figsize=(12,3.5), ylabel = r'W_' + str(h + 1), title=' ', ticks_every = 1,ticks_labels_size=sl,title_size=st)    
plt.tight_layout()

namefig = out_fold + '/weights.png'
fig.savefig(namefig,  bbox_inches = 'tight', pad_inches = 0, dpi = 300)
plt.close()

colors = ['r','orange', 'deepskyblue', 'blue', 'springgreen', 'forestgreen']

s2 = 18
sl = 12
list_ix=[2,6] ## example of hidden units, but any could be selected ##
list_iy=[8] ## example of hidden units, but any could be selected ##
s1 = 1.0

for ix in list_ix:
    for iy in list_iy:
        fig, ax = plt.subplots()
        fig.set_figwidth(5)
        fig.set_figheight(5)

        for h in range(nc):
            sel_n = butils.convert_number(seqs_all[h])
            In = RBM.vlayer.compute_output(sel_n, RBM.weights)
            ax.scatter(In[:,ix],In[:,iy], c = colors[h], s = s1, label = hlas[h])

        ax.set_xlabel('Input to h_' + str(ix+1), fontsize=s2)
        ax.set_ylabel('Input to h_' + str(iy+1), fontsize=s2)
        ax.tick_params(axis='both', which='major', labelsize = s2)
        plt.tight_layout()
        plt.legend(fontsize=10,markerscale=2,frameon=False, loc ='upper right')
        pathfig = out_fold + '/Input_' + str(ix+1) + '_vs_Input_' + str(iy+1) + '.png'
        fig.savefig(pathfig,  bbox_inches = 'tight', pad_inches = 0, dpi = 300)
        plt.close()

Here, we plot inputs to hidden unit 10 which we found 
enables the discrimination of HLA-B51:01 binding modes, as shown in Fig. 2c.

In [99]:
# tools for aminoacids
aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V',  'W', 'Y','-']
aadict_rev = {k:aa[k] for k in range(len(aa))}
aadict = {aa[k]: k for k in range(len(aa))} 

cy = ['C']
group_cy = [aadict[i] for i in cy]
pol = ['Q','N','S','T']
group_pol = [aadict[i] for i in pol]
pos = ['K','R','H']
group_pos = [aadict[i] for i in pos]
neg = ['D','E']
group_neg = [aadict[i] for i in neg]
aro = ['F','W','Y'];
group_aro = [aadict[i] for i in aro]
ali = ['V','L','I','M'];
group_ali = [aadict[i] for i in ali]
tiny = ['A','P','G'];
group_tiny = [aadict[i] for i in tiny]
gap = ['-']
group_gap = [aadict[i] for i in gap]

hydrophobic = ['I', 'L', 'V', 'M', 'F', 'Y', 'W','P']
hydrophilic = ['A', 'G', 'T', 'S', 'N', 'Q', 'D', 'E', 'R']
group_hydrophilic = [aadict[i] for i in hydrophilic]
group_hydrophobic = [aadict[i] for i in hydrophobic]

prop = ['tiny','cy','neg','neg','aro','tiny','pos','ali','pos','ali','ali','pol','tiny','pol','pos','pol','pol','ali','aro','aro','gap']
aadict_prop = {k:prop[k] for k in range(len(prop))}

ind_hla = 3 # 'HLA-B*51:01'
sel_n = butils.convert_number(seqs_orig[ind_hla])

ind_1 = [] # Hydrophobic
for i in range(len(sel_n)):
    if sel_n[i][0] in group_hydrophobic and sel_n[i][1] in group_hydrophobic:
        ind_1.append(i) # select sequences that have 'R' in position 3       
ind_2 = [] # hydrophilic + charge
for i in range(len(sel_n)):
    if (aadict_prop[sel_n[i][0]]=='neg' or aadict_prop[sel_n[i][0]]=='pol') and sel_n[i][1] in group_hydrophilic:
        ind_2.append(i)   

fig, ax1 = plt.subplots()
fig.set_figwidth(5)
fig.set_figheight(5)
ind_h = 9 # 'HLA-B*51:01' 

In = RBM.vlayer.compute_output(sel_n, RBM.weights)

ax1.hist(In[ind_1,ind_h],weights=None, bins=100, color = 'DimGrey', label = 'Hydrophilic/charge')
ax1.hist(In[ind_2,ind_h],weights=None, bins=100, color = 'purple', label = 'Hydrophobic')
plt.legend(fontsize=sl,markerscale=4,frameon=False, loc = 'upper left')
ax1.tick_params(axis='both', which='major',labelsize = sl+2)
ax1.set_xlabel('Input to h_' + str(ind_h + 1),fontsize=sl+2)
ax1.set_ylabel('Count',fontsize=sl+2)
pathfig = out_fold + '/hist_Input_' + str(ind_h+1) + '.png'
fig.savefig(pathfig)
plt.close()

**Ranking neoantigens**:
Here, we read the tables with scores assigned to mutant peptides in the patient's WES and we find the rank associated to MS-validated neoantigens, as in Supplementary Table 2.

In [100]:
### Read different tables ##
out_fold_NA = out_fold + '/WESranking' 
na_na = ['DANSFLQSV','GVYPMPGTQK']
na_hla = ['HLA-B*51:01','HLA-A*03:01']

fn = out_fold_NA + '/WES_YesReal_1_hu_10_l120.001_AL9.5_19_SA9_RWNoAlpha_TR1.txt'
wes_out = pd.read_csv(fn, sep = '\t')
fn = out_fold_NA + '/WES_YesReal_1_hu_10_l120.001_AL9.5_19_SA9_RWNoAlpha_TR1_pred.txt'
predvec = np.loadtxt(fn)
fnkm = out_fold_NA + '/WES_YesReal_1_hu_10_l120.001_AL9.5_19_SA9_RWNoAlpha_TR1_pred_kmeans.txt'
predveckm = np.loadtxt(fnkm)

wespeps = list((wes_out['wes']).values)
wespeps_g = list((wes_out['wes_g']).values)
likwes = list((wes_out['LL']).values)
lenwes = list((wes_out['Len']).values)
predwes = list((wes_out['pred']).values)
predwesl = list((wes_out['pred_label']).values)
score_mixmhc = list((wes_out['score_mixmhc']).values)
val_score_mixmhc = list((wes_out['val_score_mixmhc']).values)
val_score_pwmrbm = list((wes_out['val_score_pwmrbm']).values)
score_kmeans = list((wes_out['labels_kmeans']).values)
val_score_kmeans = list((wes_out['val_score_kmeans']).values)

ent = []
for t in range(len(predvec)):
    ent.append(-sum(predvec[t]*np.log(predvec[t]+0.000001)))

entkm = []
for t in range(len(predveckm)):
    entkm.append(-sum(predveckm[t]*np.log(predveckm[t]+0.000001)))

## define a presentation score ##
likwes_f = likwes - np.array(ent)
kmeans = likwes - np.array(entkm)

NAlist = zip(wespeps, wespeps_g, likwes_f, lenwes, predwes)
ind_ll = 2
ind_len = 3
NArankL = sorted(sorted(NAlist, key = lambda x : x[ind_len]), key = lambda x : x[ind_ll], reverse = True)
rankedwes = list(np.array(NArankL)[:,0])

rankll=[]
for t in range(len(rankedwes)):
    rankll.append(wespeps.index(rankedwes[t]))

NAlist = zip(wespeps, wespeps_g, val_score_mixmhc, lenwes, predwes)
NArankM = sorted(sorted(NAlist, key = lambda x : x[ind_len]), key = lambda x : x[ind_ll], reverse = True)
rankedwesm = list(np.array(NArankM)[:,0])
rankmm=[]
for t in range(len(rankedwesm)):
    rankmm.append(wespeps.index(rankedwesm[t]))

NAlist = zip(wespeps, wespeps_g, kmeans, lenwes, predwes)
NArankK = sorted(sorted(NAlist, key = lambda x : x[ind_len]), key = lambda x : x[ind_ll], reverse = True)
rankedwesk = list(np.array(NArankK)[:,0])
rankkm=[]
for t in range(len(rankedwesk)):
    rankkm.append(wespeps.index(rankedwesk[t]))

## perform ranking for RBM-MHC and RBM-km ##
lna=[]
lhla=[]
lrbm_hla=[]
lrbm_lik=[]
lrbm_rank=[]
lkm_hla=[]
lkm_score=[]
lkm_rank=[]
km_score=[]
rbm_score=[]
for t in range(len(na_na)):
    if na_na[t] in wespeps:  
        ind = wespeps.index(na_na[t])
        lna.append(na_na[t])
        lhla.append(na_hla[t])
        # pred RBM
        lrbm_hla.append(na_hla[t] == hlas[predwesl[ind]])          
        lkm_hla.append(na_hla[t] == hlas[score_kmeans[ind]])
        lkm_score.append(format(val_score_kmeans[ind], '.3f'))
        lrbm_lik.append(format(likwes[ind]/np.log(10), '.3f'))
        km_score.append(likwes_f[ind])
        rbm_score.append(kmeans[ind])
        lrbm_rank.append(list(np.array(wespeps)[rankll]).index(na[t])+1)
        lkm_rank.append(list(np.array(wespeps)[rankkm]).index(na[t])+1)

title_text = 'Neoantigen prediction 12T (total candidates = ' + str(len(wespeps))+ ')'
labels_rank = ['Neoantigen', 'Rank RBM-MHC', 'HLA-I predicted?', 'Rank RBM-kM', 'HLA-I predicted?' ]
values = np.array([lna,lrbm_rank, lrbm_hla,lkm_rank, lkm_hla,])

# Build a dataframe with ranking #

from pandas.plotting import table

df = pd.DataFrame(np.transpose(values), columns=labels_rank)
fig, ax = plt.subplots(figsize = (2*len(values),len(na_na))) # set size frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
tabla = table(ax, df, loc='center')  # where df is your data frame
tabla.auto_set_font_size(False) # Activate set fontsize manually
tabla.set_fontsize(12)
plt.suptitle(title_text, fontsize = 15)
fig.show()
pathfig = out_fold_NA + '/TableNArank.pdf'
fig.savefig(pathfig)

/home/barbara/.local/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
